In [1]:
import os
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt

In [2]:
# read nuts shp and enter the path for
nuts = gpd.read_file('/data/xiang/1-Data/NUTS/NUTS_RG_01M_2021_3035.shp')
read_folder = '/data/xiang/1-Data/Urban atlas'

In [ ]:
for folder in os.listdir(read_folder):
    # get the country id and ua id of each shp
    country_id = folder.split('_')[0][:2]
    ua_num = folder.split('_')[0][2:5]

    # read the shp and separate green area from the total area
    sub_folder = read_folder + r'/' + folder + r'/' + 'Data' + r'/'
    ua = gpd.read_file(sub_folder + os.listdir(sub_folder)[0])
    ua.to_crs('EPSG:3035', inplace = True)
    green = ua[ua['class_2018']=='Green urban areas']
    
    # get the total area and total green area of shp that falls within the boundary of NTUS regions in the same country
    country_shp = nuts[nuts['NUTS_ID'].str.contains(country_id)]
    dict_temp = {}
    for subregion_id in country_shp['NUTS_ID']:
        subregion = country_shp[country_shp['NUTS_ID']==subregion_id]
        Green_temp = gpd.sjoin(green, subregion, how='inner')
        Green_temp['shape area'] = Green_temp.area
        total_temp = gpd.sjoin(ua, subregion, how='inner')
        total_temp['shape area'] = total_temp.area
        dict_temp[subregion_id] = [Green_temp['shape area'].sum(),total_temp['shape area'].sum()]

    # record the green and total area and export the dataframe
    df_temp = pd.DataFrame([dict_temp]).T
    df_temp['green area'] = df_temp[0].apply(lambda x: x[0])
    df_temp['total area'] = df_temp[0].apply(lambda x: x[1])
    df_temp[['green area','total area']].to_excel('/data/xiang/3-case studies/1-urban atlas/FUAs' + r'/' + country_id + '_' + ua_num + '.xlsx')

In [ ]:
file_list = os.listdir('/data/xiang/3-case studies/1-urban atlas/FUAs')
for country in country_list:
    file_country_list = [i for i in file_list if i.split('_')[0] == country]
    df_comb = None
    for file in file_country_list:
        # for FUAs in the same country, sum the total of green and total areas to each NUTS level
        df_temp = pd.read_excel('/data/xiang/3-case studies/1-urban atlas/FUAs' + r'/' + file, index_col=0)
        df_comb = df_temp if df_comb is None else df_comb + df_temp
    df_comb.to_excel('/data/xiang/3-case studies/1-urban atlas/country merge' + r'/' + country + '.xlsx')

In [ ]:
df_concat = None
for file in os.listdir('/data/xiang/3-case studies/1-urban atlas/country merge'):
    df_temp = pd.read_excel('/data/xiang/3-case studies/1-urban atlas/country merge' + r'/' + file)
    df_concat = df_temp if df_concat is None else pd.concat([df_concat, df_temp])

In [ ]:
df_concat.columns = ['geo', 'obsValue']
df_concat['obsTime'] = '2018'
df_concat['unit'] = 'square meters'
df_concat['freq'] = '6 years'
df_concat.to_excel('/data/xiang/3-case studies/1-urban atlas/Urban green area.xlsx')

In [ ]:
df_concat